In [ ]:
import numpy 
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt

import torchvision
import torchvision.transforms as tf

In [ ]:
train_path=r"training dataset path"
test_path=r"testing dataset path"

In [ ]:
Train_transforms= tf.Compose([
        tf.Resize([140,116]),tf.RandomHorizontalFlip(),tf.RandomRotation(5),tf.ToTensor(),tf.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
Test_transforms=tf.Compose([
        tf.Resize([140,116]), tf.ToTensor(),tf.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_dataset= torchvision.datasets.ImageFolder(root=train_path, transform=Train_transforms)
test_dataset= torchvision.datasets.ImageFolder(root=test_path, transform=Test_transforms)

In [ ]:
train_loader= torch.utils.data.DataLoader(dataset=train_dataset,batch_size=1,shuffle=True)
test_loader= torch.utils.data.DataLoader(dataset=train_dataset,batch_size=1,shuffle=False)


In [ ]:
image,label=train_dataset[0]

image.size()

In [ ]:
class_names=['Fake Oracle Charachter','Real Oracle Charachter','Roboccon Logo']

In [ ]:
class Convolutional_Layer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1= nn.Conv2d(3,12,3)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(12,24,3)
        self.full1=nn.Linear(21384,120)
        self.full2=nn.Linear(120,84)
        self.full3=nn.Linear(84,3)

    def forward(self,x):
        x= self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x= torch.flatten(x,1)
        x= F.relu(self.full1(x))
        x= F.relu(self.full2(x))
        x= self.full3(x)
        return x   

In [ ]:
net= Convolutional_Layer()
loss_function=nn.CrossEntropyLoss()
optimizer=opt.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [ ]:
for epoch in range(70):
    print(f"Training epoch {epoch}...")

    running_loss=0.0

    for i,data in enumerate(train_loader):
        inputs,labels=data

        optimizer.zero_grad()
        outputs=net(inputs)

        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

    print(f'loss: {running_loss/len(train_loader):.4f}')

In [ ]:
torch.save(net.state_dict(),'trained_net3.pth')